In [1]:
import pandas as pd
from apyori import apriori

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#### Load Dataset

In [2]:
df = pd.read_csv('./mercado2.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Converting to list

In [3]:
transactions = df.to_numpy().astype(str).tolist()

#### Creating the association

In [4]:
rules = apriori(transactions, min_support=0.003, min_confidence=0.2, min_lift=2)
results = [a for a in rules]

In [5]:
results[0:3]

[RelationRecord(items=frozenset({'almonds', 'burgers'}), support=0.005199306759098787, ordered_statistics=[OrderedStatistic(items_base=frozenset({'almonds'}), items_add=frozenset({'burgers'}), confidence=0.25490196078431376, lift=2.923577382023146)]),
 RelationRecord(items=frozenset({'bacon', 'spaghetti'}), support=0.003199573390214638, ordered_statistics=[OrderedStatistic(items_base=frozenset({'bacon'}), items_add=frozenset({'spaghetti'}), confidence=0.36923076923076925, lift=2.1206738131699847)]),
 RelationRecord(items=frozenset({'black tea', 'milk'}), support=0.004266097853619517, ordered_statistics=[OrderedStatistic(items_base=frozenset({'black tea'}), items_add=frozenset({'milk'}), confidence=0.29906542056074764, lift=2.3079112341833006)])]

In [6]:
df_results = pd.DataFrame(columns=['association', '_if', '_then', 'support', 'confidence', 'lift'])

for result in results:
    df_temp = pd.DataFrame(columns=['association', '_if', '_then', 'support', 'confidence', 'lift'])
    df_temp['association'] = [[r for r in result[0]]]
    df_temp['support'] = result[1]
    
    for register in result[2]:
        r = [a for a in register]
        if len(r[0]) > 1:
            df_temp['_if'] = [[a for a in r[0]]]
        else:
            df_temp['_if'] = [a for a in r[0]]
            
        if len(r[1]) > 1:
            df_temp['_then'] = [[a for a in r[1]]]
        else:
            df_temp['_then'] = [a for a in r[1]]
            
        df_temp['confidence'] = r[2]
        df_temp['lift'] = r[3]
            
        df_results = pd.concat([df_results, df_temp], ignore_index=True)

#### Showing the results

In [7]:
df_results.sort_values('lift', ascending=False)

,association,_if,_then,support,confidence,lift
1064,"[frozen vegetables, mineral water, soup, milk]","[frozen vegetables, soup]","[mineral water, milk]",0.003066,0.383333,7.987176
1653,"[nan, milk, frozen vegetables, mineral water, ...","[frozen vegetables, soup]","[mineral water, nan, milk]",0.003066,0.383333,7.987176
1658,"[nan, milk, frozen vegetables, mineral water, ...","[frozen vegetables, soup, nan]","[mineral water, milk]",0.003066,0.383333,7.987176
1059,"[frozen vegetables, mineral water, olive oil, ...","[frozen vegetables, olive oil]","[mineral water, milk]",0.003333,0.294118,6.128268
1647,"[olive oil, nan, milk, frozen vegetables, mine...","[frozen vegetables, olive oil, nan]","[mineral water, milk]",0.003333,0.294118,6.128268
...,...,...,...,...,...,...
738,"[mineral water, chocolate, nan, escalope]","[mineral water, escalope]","[chocolate, nan]",0.005599,0.328125,2.002657
156,"[mineral water, chocolate, escalope]","[mineral water, escalope]",chocolate,0.005599,0.328125,2.002657
107,"[cake, spaghetti, pancakes]","[cake, pancakes]",spaghetti,0.004133,0.348315,2.000542
636,"[cake, spaghetti, pancakes, nan]","[cake, pancakes]","[spaghetti, nan]",0.004133,0.348315,2.000542
